In [4]:
import os
import datetime 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.firefox import GeckoDriverManager
import pandas as pd

In [5]:
def catch_data(url,col_name,shift=0):
    if not os.path.exists('./web_data'):
        os.makedirs('./web_data')
    if os.path.exists('./web_data/'+col_name+'.csv'):
        data=pd.read_csv('./web_data/'+col_name+'.csv')
        return data
    
    driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    driver.get(url)
    dates = driver.execute_script("return Highcharts.charts[0].series[0].xData")
    index = driver.execute_script("return Highcharts.charts[0].series[0].yData")
    #only include the data in recent 20 years
    if len(dates)>6000:
        dates=dates[-6000:]
        index=index[-6000:]
        
    index_day=[datetime.datetime.fromtimestamp(i/1000).strftime('X%d/X%m/%Y').replace('X0','X').replace('X','') for i in dates]
    
    data=pd.DataFrame({'date':index_day,col_name:index})
    data[col_name]=data[col_name].shift(shift)
    data.to_csv('./web_data/'+col_name+'.csv',index=False)
    driver.close()
    return data

In [6]:
NYMEX_future=catch_data("https://en.macromicro.me/collections/68/copper-relative/1405/nymex-copper-future-pricer","NYMEX_future",shift=1)
Copper_COT_index=catch_data("https://en.macromicro.me/collections/68/copper-relative/942/copper-cot-index","Copper_COT_index",shift=1)
LME_Copper_Stock=catch_data("https://en.macromicro.me/collections/68/copper-relative/943/lme-copper-warehouse-stock","LME_Copper_Stock",shift=1)
Caixin_PMI=catch_data("https://en.macromicro.me/collections/68/copper-relative/964/copper-china-manufacturing-pmi","Caixin_PMI",shift=3)


[WDM] - Downloading: 16.9kB [00:00, 17.3MB/s]                   
[WDM] - Downloading: 16.9kB [00:00, 8.69MB/s]                   
[WDM] - Downloading: 16.9kB [00:00, 6.86MB/s]                   


In [9]:
pd.merge(pd.merge(NYMEX_future,Copper_COT_index,on='date'),LME_Copper_Stock,on='date')

,date,NYMEX_future,Copper_COT_index,LME_Copper_Stock
0,07/01/2002,0.6700,5674.0,800375.0
1,14/01/2002,0.7060,13604.0,807125.0
2,28/01/2002,0.7090,23165.0,830325.0
3,04/02/2002,0.7335,22237.0,855275.0
4,11/02/2002,0.7315,35246.0,866300.0
...,...,...,...,...
964,19/12/2022,3.7752,9115.0,84100.0
965,09/01/2023,3.9550,-4512.0,86400.0
966,16/01/2023,4.2050,26480.0,83850.0
967,23/01/2023,4.2960,38933.0,80025.0


In [19]:
merged_data=pd.merge(pd.merge(pd.merge(NYMEX_future,Copper_COT_index,on='date',how='left'),LME_Copper_Stock,on='date',how='left'),Caixin_PMI,on='date',how='left')
merged_data.fillna(method='ffill',inplace=True)
merged_data.to_csv('MM.csv',index=False)